In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model



from keras.optimizers import Adam
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Input, Dropout, Conv2D,Activation
from keras.utils import np_utils
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

Using TensorFlow backend.
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/t

In [2]:
dirs = os.listdir("./data/")
print (dirs)

['horses', 'cats', 'dogs', 'Humans']


In [3]:
path = "./data/"

In [4]:
images = []
labels = []

In [5]:
label_dicts = {"horses":0, "cats":1, "dogs":2, "Humans":3}

In [6]:
for ix in dirs:
    class_path = path + ix + "/"
    img_names = os.listdir(class_path)
    for im in img_names:
        im = image.load_img(class_path + im, target_size=(224,224))
        im_array = image.img_to_array(im)
        images.append(im_array)
        labels.append(label_dicts[ix])
        
print (len(images), len(labels))

808 808


In [7]:
combined = list(zip(images,labels))
random.shuffle(combined)

images[:],labels[:] = zip(*combined) 

In [8]:
def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Activation(swish)})

W0529 15:01:28.388687 139731239167808 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [9]:
print(labels[:10])

[1, 1, 3, 0, 1, 3, 0, 3, 1, 2]


In [10]:
X_train = np.array(images)
y_train = np.array(labels)
y_train = np_utils.to_categorical(y_train)
print (X_train.shape, y_train.shape)

(808, 224, 224, 3) (808, 4)


In [ ]:
res_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

W0529 15:01:31.497211 139731239167808 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0529 15:01:31.501898 139731239167808 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0529 15:01:31.523763 139731239167808 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0529 15:01:31.524362 139731239167808 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.Co

In [ ]:
res_model.summary()

In [ ]:
## model1
c1 = Conv2D(1, (3,3), activation='swish')(res_model.output)
f1 = Flatten()(c1)
fc1 = Dense(4, activation='softmax')(f1)

model = Model(inputs=res_model.inputs, outputs=fc1)
model.summary()

In [ ]:
## Model2
avg = GlobalAveragePooling2D()(res_model.output)
fc1 = Dense(256, activation='swish')(avg)
fc2 = Dense(4, activation='softmax')(fc1)

model = Model(inputs=res_model.inputs, outputs=fc2)
model.summary()

In [ ]:
for ix,layers in enumerate(model.layers):
    print (ix, layers)

In [ ]:
for ix in range(171):
    model.layers[ix].trainable = False

In [ ]:
model.summary()

In [ ]:
adam = Adam(lr=0.00003)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train,y_train,epochs=5,shuffle=True,batch_size=64,validation_split=0.20)

In [ ]:
## No_finetuning

plt.figure(0)
plt.plot(hist.history['acc'],'r')
plt.plot(hist.history['val_acc'],'b')

plt.figure(1)
plt.plot(hist.history['loss'],'r')
plt.plot(hist.history['val_loss'],'b')

plt.show()

In [ ]:
## finetuning

plt.figure(0)
plt.plot(hist.history['acc'],'r')
plt.plot(hist.history['val_acc'],'b')

plt.figure(1)
plt.plot(hist.history['loss'],'r')
plt.plot(hist.history['val_loss'],'b')

plt.show()